In [2]:
!pip install selenium

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [45]:
!pip3 install bs4

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/66/25/ff030e2437265616a1e9b25ccc864e0371a0bc3adb7c5a404fd661c6f4f6/beautifulsoup4-4.9.1-py3-none-any.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/6f/8f/457f4a5390eeae1cc3aeab89deb7724c965be841ffca6cfca9197482e470/soupsieve-2.0.1-py3-none-any.whl
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /home/azlaan/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [6]:
!pip install stem

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.9 MB 1.8 MB/s eta 0:00:01
  Created wheel for stem: filename=stem-1.8.0-py2-none-any.whl size=434489 sha256=62fe852ebd29d996e9f23bb8bee50325a788b9a37d27a326c64aceb275ce22b7
  Stored in directory: /home/azlaan/.cache/pip/wheels/aa/ba/df/97ea10d90972f4b84cc82cb97f5959972860397aca3334f1b1
Successfully built stem
You should consider upgrading via the '/usr/bin/p

In [39]:
!pip3 install lxml

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 5.5MB 223kB/s ta 0:00:011


In [41]:
!pip install lxml

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from stem import Signal
from stem.control import Controller
import time
import logging
import threading
import csv
import os

In [8]:
def switchIP():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

In [21]:
def my_proxy(PROXY_HOST,PROXY_PORT):
    fp = webdriver.FirefoxProfile()
    # Direct = 0, Manual = 1, PAC = 2, AUTODETECT = 4, SYSTEM = 5
    fp.set_preference("network.proxy.type", 1)
    fp.set_preference("network.proxy.socks",PROXY_HOST)
    fp.set_preference("network.proxy.socks_port",int(PROXY_PORT))
    fp.update_preferences()
    options = Options()
    options.headless = True
    path = os.path.join(os.getcwd(), 'geckodriver')
    return webdriver.Firefox(options=options, firefox_profile=fp, executable_path=path)

In [42]:
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(1):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time + (0.01*i))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        #print(i, end='\r')
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [43]:
def all_links(url):
    print('Thread {} started.'.format(threading.current_thread().name))
    # Setup the driver. This one uses firefox with some options and a path to the geckodriver
    driver = my_proxy("127.0.0.1", 9050)
    # implicitly_wait tells the driver to wait before throwing an exception
    driver.implicitly_wait(30)
    # driver.get(url) opens the page
    driver.get(url)
    # This starts the scrolling by passing the driver and a timeout
    scroll(driver, 5)
    # Once scroll returns bs4 parsers the page_source
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
    # Then we close the driver as soup_a is storing the page source
    driver.close()
    header =['Title', 'Link']
    # Empty array to store the links
    links = []
    with open('{}.csv'.format(threading.current_thread().name), mode='a') as link_dump:
        link_writer = csv.writer(link_dump, delimiter=',')
        link_writer.writerow(header)
        # Looping through all the h1 elements in the page source that contain the classes seen below
        for heading in soup_a.find_all('div', {"class":"fv fw"}):
            # link.get('href') gets the href/url out of the h1 element
            a = heading.find('a')
            title = a.get_text()
            link = a.get('href')
            link_init = "https://medium.com"
            if link.find('https://', 0, 9) == 0:
                link_writer.writerow([title, link])
            else:
                link = link_init + link
                link_writer.writerow([title, link])     

    return

In [46]:
# Medium Topic URLs
url_ai = 'https://medium.com/topic/artificial-intelligence'
url_ds = 'https://medium.com/topic/data-science'
url_ml = 'https://medium.com/topic/machine-learning'

# Init threads
thread_ai = threading.Thread(target=all_links, name='thread_ai', args=(url_ai,))
'''thread_ds = threading.Thread(target=all_links, name='thread_ds', args=(url_ds,))
thread_ml = threading.Thread(target=all_links, name='thread_ml', args=(url_ml,))'''

# Start threads
thread_ai.start()
#thread_ds.start()
#thread_ml.start()

# Wait for threads to finish
thread_ai.join()
#thread_ds.join()
#thread_ml.join()

Thread thread_ai started.


Exception in thread thread_ai:
Traceback (most recent call last):
  File "/usr/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/usr/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-43-383843b7aa95>", line 12, in all_links
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
  File "/home/azlaan/.local/lib/python2.7/site-packages/bs4/__init__.py", line 245, in __init__
    % ",".join(features))
FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?



In [63]:
import requests
url = 'https://www.udacity.com/courses/school-of-ai'

In [64]:
source = requests.get(url)

In [68]:
soup = BeautifulSosource.content

'<!DOCTYPE html><html><head>\n  <meta charset="utf-8">\n  <script type="text/javascript" class="ng-star-inserted">window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(18),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",function(t){r(t.stack)}),s.dev&&i.on("fn-err",function(t,n,e){r(e.stack)}),s.dev&&(r("NR AGENT IN DEVELOPMENT MODE"),r("flags: "+a(s,function(t,n){return t}).join(", ")))},{}],2:[function(t,n,e){function r(t,n,e,r,s){try{p?p-=1:o(s||new UncaughtException(t,

In [50]:
len(dat)

62

In [51]:
dat


,Title,Link
0,Title,Link
1,Facial Recognition Is Law Enforcement’s Newest...,https://onezero.medium.com/facial-recognition-...
2,Facial Recognition Is Law Enforcement’s Newest...,https://onezero.medium.com/facial-recognition-...
3,Instilling Responsible and Reliable AI Develop...,https://medium.com/accenture-the-dock/instilli...
4,AI & Academic Search Engines,https://medium.com/dataseries/ai-academic-sear...
5,Making AI Fair,https://medium.com/ai-in-plain-english/making-...
6,12 Main Dropout Methods : Mathematical and Vis...,https://towardsdatascience.com/12-main-dropout...
7,The New Art of AI Engineering,https://medium.com/@wouterhuygen/the-new-art-o...
8,GPT-3 for the People,https://towardsdatascience.com/gpt-3-for-the-p...
9,A Strategic look at Robotic Process Automation.,https://medium.com/basicolans/a-strategic-look...


In [60]:
dat.loc[1]['Title']

'Facial Recognition Is Law Enforcement\xe2\x80\x99s Newest Weapon Against Protesters'

In [61]:
dat.loc[2]['Title']

'Facial Recognition Is Law Enforcement\xe2\x80\x99s Newest Weapon Against Protesters'

In [70]:
dat = pd.read_pickle('udacity_ai.pkl')

ValueError: unsupported pickle protocol: 4